In [ ]:
import tvm

from tvm.script import ir as I
from tvm.script import tir as T
from tvm.script import relax as R


def showmod(mod: tvm.ir.module.IRModule):
    mod.show(
        black_format=True,
        show_meta=False,
        verbose_expr=True,
        show_object_address=False,
        show_all_struct_info=True,
    )


def createandshowmod(ops):
    te_func = te.create_prim_func(ops).with_attrs({"global_symbol": "test"})
    mod = tvm.IRModule({"test": te_func})
    showmod(mod)


from tvm.topi.transform import *

## expand_dims

In [ ]:
A: te.Tensor = te.placeholder((128, 128), name="A")

expand_dims_tensor = expand_dims(a=A, axis=0, num_newaxis=2)
createandshowmod([A, expand_dims_tensor])

expand_dims_tensor = expand_dims(a=A, axis=1, num_newaxis=2)
createandshowmod([A, expand_dims_tensor])

expand_dims_tensor = expand_dims(a=A, axis=-1, num_newaxis=2)
createandshowmod([A, expand_dims_tensor])

## expand_like

In [ ]:
"""Expand an input array with the shape of second array.
This operation can always be composed of unsqueezing and
expanding dims on those unsqueezed axes.

Parameters
----------
a : tvm.te.Tensor
    The tensor to be expanded.
shape_like : tvm.te.Tensor
    The tensor to with target shape.
axis: list of int
    axis to be expanded on

Returns
-------
ret : tvm.te.Tensor
"""

print(expand_dims_tensor)

"""The shape of output tensor is the same as the shape of `shape_like`.

`axis` determines the axis of `A` that will be expanded. `axis` should
be a list of axis, and len(axis) + len(A.shape) = len(shape_like.shape).
`axis` can be negative, which means counting dimensions from the back.

For example, if A.shape = [128, 128, 256, 256], shape_like.shape =
[128, 128, 128, 128], axis = [1, 2], then the shape of output tensor is
[128, 128, 128, 128]. `axis` means the axis [1, 2] of `A` will be expanded.
And it will read A[0, 3] and write to output.
"""
A = te.placeholder(shape=(256, 256), dtype="int32", name="A")
expand_dims_tensor = te.placeholder(
    shape=(128, 128, 128, 128), dtype="int32", name="expand_dims_tensor"
)

expand_like_tensor = expand_like(a=A, shape_like=expand_dims_tensor, axis=[1, 2])
createandshowmod([A, expand_like_tensor])

expand_like_tensor = expand_like(a=A, shape_like=expand_dims_tensor, axis=[0, 1])
createandshowmod([A, expand_like_tensor])

## transpose

In [ ]:
"""Permute the dimensions of an array.

Parameters
----------
a : tvm.te.Tensor
    The tensor to be expanded.

axes: tuple of ints, optional
    By default, reverse the dimensions.

Returns
-------
ret : tvm.te.Tensor
"""

A: te.Tensor = te.placeholder(shape=(256, 256, 256), dtype="float32", name="A")
createandshowmod([A, transpose(a=A, axes=(1, 0, 2))])

## flip

In [ ]:
"""Flip/reverse elements of an array in a particular axis.

Parameters
----------
a : tvm.te.Tensor
    The tensor to be expanded.

axis : int, optional
    The axis along which the tensors will be reveresed.

Returns
-------
ret : tvm.te.Tensor
"""

createandshowmod([A, flip(A, axis=0)])
createandshowmod([A, flip(A, axis=1)])

## reverse_sequence

In [ ]:
"""Reverse the tensor for variable length slices.
Input is first sliced along batch axis and then elements are reversed along seq axis.

Parameters
----------
a : tvm.te.Tensor
   The tensor to be reversed.

seq_lengths : tvm.te.Tensor
   A 1D Tensor with length a.dims[batch_axis]
   Must be one of the following types: int32, int64
   if seq_lengths[i] > a.dims[seq_axis], it is rounded to a.dims[seq_axis]
   if seq_lengths[i] < 1, it is rounded to 1

seq_axis : int, optional
   The axis along which the elements will be reversed. Default is 1.

batch_axis : int, optional
   The axis along which the tensor will be sliced. Default is 0.

Returns
-------
ret : tvm.te.Tensor
   The computed result of same shape and type as of input.

"""

A: te.Tensor = te.placeholder(shape=(1, 2, 3), dtype="float32", name="A")
# TODO
seq_lengths: te.Tensor = te.placeholder(shape=(1), dtype="int64", name="seq_lengths")
createandshowmod([A, seq_lengths, reverse_sequence(A, seq_lengths, 1, 0)])

## strided_slice

In [ ]:
"""Slice of an array.

Parameters
----------
a : tvm.te.Tensor
    The tensor to be sliced.

begin : list of int
    The indices to begin with in the slicing.

end : list of int
    Indices indicating end of the slice.

strides : list of int, optional
    Specifies the stride values, it can be negative
    in that case, the input tensor will be reversed
    in that particular axis.

axes : list of int, optional
    Axes along which slicing is applied. When it is specified, begin, end
    strides, and axes need to a list of integers of the same length.

slice_mode : str, optional
    The slice mode [end, size].
    end - The ending indices for the slice [default].
    size - The input strides will be ignored, input end in this mode indicates
    the sizeof a slice starting at the location specified by begin. If end[i]
    is -1, all remaining elements in that dimension are included in the slice.

assume_inbound: bool, optional
    A flag to indicate if all indices are assumed to be inbound

Returns
-------
ret : tvm.te.Tensor
"""

# TODO